# A Simple Mistral 7B RAG with Weaviate Vector DB

<img src="../images/simple-rag-1-db.webp" width=700>

### 1) Load the embedding model and LLM:

In [1]:
from llama_index.core import Settings, PromptTemplate
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

import torch

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        # Since we are using a small GPU with limited memory
        # Set to False if you have a large GPU to speed things up.
        "offload_buffers": True, 
    }
)

Settings.chunk_size = 512

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### 2) Set up vector database

In [6]:
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import StorageContext

resource_owner_config = weaviate.AuthClientPassword(
    username="abc",
    password="xyz",
)

client = weaviate.Client(
    "https://my-test-3lb8837z.weaviate.network",
    auth_client_secret=resource_owner_config,
)

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex"
)

### 3) Load data / read in documents

In [8]:
from pathlib import Path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

data_dir =  Path("..") / "data" / "example-1"

documents = SimpleDirectoryReader(data_dir).load_data()

# Sanity checks
unique_docs = set(d.metadata["file_name"] for d in documents)
print(f"Read documents: {unique_docs}")

Read documents: {'Basic-Scientific-Food-Preparation-Lab-Manual.txt', 'Basic-Scientific-Food-Preparation-Lab-Manual.pdf'}


In [9]:
# This populates the database and may take a few minutes
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### 4) Set up custom prompt template

In [10]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine = index.as_query_engine()
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

### 5) Query the vector database

In [11]:
response = query_engine.query("What is dehydration?")
print(response)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
response = query_engine.query("How many tbsp. butter to use for the Cream Puffs?")
print(response)
# Correct answer is 2

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: 2 tbsp. butter are required for the Cream Puffs.


In [ ]:
response = query_engine.query("How many tbsp. butter to use for the Cream Puffs?")
print(response)
# Correct answer is 2

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: 2 tbsp. butter are required for the Cream Puffs.


In [ ]:
response = query_engine.query(
    "What are the toppings for the Braided Bread "
    "of the Braids, Coffeecake, and Sweet Rolls recipe?"
)
print(response)
# Correct answer is 
# 2 tsp. caraway seeds and ½ cup shredded Cheddar cheese.
# ½ cup diced Swiss cheese and paprika.
# ...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The toppings for the Braided Bread of the Braids, Coffeecake, and Sweet Rolls recipe are:

1. 2 tsp. caraway seeds and ½ cup shredded Cheddar cheese.
2. ½ cup diced Swiss cheese and paprika.
